<a href="https://colab.research.google.com/github/martinopiaggi/summarize/blob/main/Martino_Summarize_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summarization notebook with AIs

---



Repository: https://github.com/martinopiaggi/summarize

In [56]:
Source = "https://www.youtube.com/watch?v=3Oij7CJwfpM&list=WL&index=4" #@param {type:"string"}
Type_of_source = "Youtube video or playlist" #@param ['Text', 'Youtube video or playlist', 'Videos on Google Drive folder','Dropbox video link']

Type = Type_of_source

if Type == 'Youtube video or playlist':
  URL = Source

if Type == 'Dropbox video link':
  dropbox_URL = Source

#@markdown ---
#@markdown If source is video, you want timestamps in final summary?
Timestamps = True #@param {type:"boolean"}
#@markdown Desired output length as percentual of original length
Min_percentual_summary = 0.2 #@param {type:"number"}
Max_percentual_summary = 0.6 #@param {type:"number"}


In [ ]:
!pip install transformers
!pip install tensorflow
from transformers import pipeline,BartTokenizer, BartForConditionalGeneration
from torch.utils.data import Dataset, DataLoader

import re
import math

if Type == ("Youtube video or playlist"
            or 'Videos on Google Drive folder'
            or "Dropbox video link"):

  !pip install faster-whisper
  from faster_whisper import WhisperModel
  from pathlib import Path
  import subprocess
  import torch
  import shutil
  import numpy as np

  if Type == "Youtube video or playlist":
    !pip install yt-dlp
    from pathlib import Path
    import yt_dlp

  if Type == ("Dropbox video link"):
    !sudo apt update && sudo apt install ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [57]:
if Type == "Youtube video or playlist":
  video_path_local_list = []

  ydl_opts = {
  'format': 'm4a/bestaudio/best',
  'outtmpl': '%(id)s.%(ext)s',
  # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
  'postprocessors': [{  # Extract audio using ffmpeg
  'key': 'FFmpegExtractAudio',
  'preferredcodec': 'wav',
  }]
  }

  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download([URL])
    list_video_info = [ydl.extract_info(URL, download=False)]

  for video_info in list_video_info:
    video_path_local_list.append(Path(f"{video_info['id']}.wav"))

  #for video_path_local in video_path_local_list:
   # if video_path_local.suffix == ".mp4":
    #    video_path_local = video_path_local.with_suffix(".wav")
    #result  = subprocess.run(["ffmpeg", "-i", str(video_path_local.with_suffix(".mp4")), "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", str(video_path_local)])


if Type == ("Dropbox video link"):
    !wget -O dropbox_video.mp4 $dropbox_URL
    !ffmpeg -i dropbox_video.mp4 -vn -acodec pcm_s16le -ar 16000 -ac 1 dropbox_video_audio.wav


[youtube:tab] Extracting URL: https://www.youtube.com/watch?v=3Oij7CJwfpM&list=WL&index=4
[youtube:tab] Downloading playlist WL - add --no-playlist to download just the video 3Oij7CJwfpM
[youtube:tab] WL: Downloading webpage


[youtube] Extracting URL: https://www.youtube.com/watch?v=3Oij7CJwfpM
[youtube] 3Oij7CJwfpM: Downloading webpage
[youtube] 3Oij7CJwfpM: Downloading ios player API JSON
[youtube] 3Oij7CJwfpM: Downloading android player API JSON
[youtube] 3Oij7CJwfpM: Downloading m3u8 information
[info] 3Oij7CJwfpM: Downloading 1 format(s): 140
[download] Destination: 3Oij7CJwfpM.m4a
[download] 100% of   15.36MiB in 00:00:00 at 15.54MiB/s  
[FixupM4a] Correcting container of "3Oij7CJwfpM.m4a"
[ExtractAudio] Destination: 3Oij7CJwfpM.wav
Deleting original file 3Oij7CJwfpM.m4a (pass -k to keep)
[youtube:tab] Extracting URL: https://www.youtube.com/watch?v=3Oij7CJwfpM&list=WL&index=4
[youtube:tab] Downloading playlist WL - add --no-playlist to download just the video 3Oij7CJwfpM
[youtube:tab] WL: Downloading webpage


[youtube] Extracting URL: https://www.youtube.com/watch?v=3Oij7CJwfpM
[youtube] 3Oij7CJwfpM: Downloading webpage
[youtube] 3Oij7CJwfpM: Downloading ios player API JSON
[youtube] 3Oij7CJwfpM: Downloading android player API JSON
[youtube] 3Oij7CJwfpM: Downloading m3u8 information


In [61]:
language = "en"
initial_prompt = ""
Text = ""

video_path_local = str(video_path_local_list[0])

if Type == ("Dropbox video link"):
    video_path_local = "dropbox_video_audio.wav"


if Type is not ("Text" or "Text from Google Drive"):

  model = WhisperModel('small', device="cuda", compute_type='int8')
  segments, info = model.transcribe(str(video_path_local), beam_size=5,
                                    language=None if language == "auto" else language,
                                    task="translate",
                                    initial_prompt=initial_prompt)

  transcript_file_name = video_path_local.replace(".wav", ".txt")
  with open(transcript_file_name, 'w') as f:
    for segment in segments:
      Text += segment.text.strip()
    f.write(Text+". ")


In [62]:
#Clean cache to avoid Colab crash in case of multiple runs
torch.cuda.empty_cache()
#Whisper model no longer necessary
del model
#load model
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name).to("cuda:0")

if Type is not ("Text" or "Text from Google Drive"):
  Text = open(transcript_file_name, "r").read()
  if Type == "Dropbox video link":
    Text = open("dropbox_video_audio.txt", "r").read()


# Define the TextDataset
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]


# Tokenize the text and split into batches
texts = [Text[i:i+512] for i in range(0, len(Text), 512)]
dataset = TextDataset(texts)
dataloader = DataLoader(dataset, batch_size=16, shuffle=False)

summary = ''

for batch in dataloader:

    # Convert the batch to the appropriate format
    encoded_inputs = tokenizer(batch, truncation=True, padding=True, return_tensors="pt", max_length=512).to("cuda:0")

    # Determine max_length and min_length based on input length
    min_length = int(Min_percentual_summary * encoded_inputs["input_ids"].size(1))
    max_length = int(Max_percentual_summary * encoded_inputs["input_ids"].size(1))

    summaries_output = model.generate(
        input_ids=encoded_inputs["input_ids"],
        attention_mask=encoded_inputs["attention_mask"],
        num_beams=4,
        length_penalty=0.6,
        no_repeat_ngram_size=2,
        min_length=min_length,  # Set the min_length
        max_length=max_length   # Set the max_length
    )


    # Decode each summarized output and append to the final summary
    for idx, output in enumerate(summaries_output):
        decoded_summary = tokenizer.decode(output, skip_special_tokens=True)
        summary += decoded_summary

# Save the summary

final_name = 'summary_'
if Type is not ("Text" or "Text from Google Drive"):
  final_name += transcript_file_name
  if Type == "Dropbox video link":
    final_name += "dropbox_video_audio.txt"

with open(final_name, 'w') as f:
    f.write(summary)